# Project 2

## Setup

In [ ]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble as ens
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
import datetime
from datetime import date, timedelta
import calendar  # Ahh, importing the weekdays

In [ ]:
# Import data from .csv file
traffic_data = pd.read_csv("data.csv")
print(traffic_data)

## Visualisation

In [ ]:
# Plot all the total traffic per hour over time to give a holistic impression of the data
y_volum_totalt = traffic_data['Volum totalt']
x_data_index = list(range(0, len(y_volum_totalt)))
fig = plt.figure(figsize=(100,10))
plt.plot(x_data_index, y_volum_totalt)
plt.show()
#It looks like there is a tendency for the traffic to slow down at around the same time each year

In [ ]:
#Let us have a look at the individual years
year_2015 = traffic_data[traffic_data['År'] == 2015]
year_2016 = traffic_data[traffic_data['År'] == 2016]
year_2017 = traffic_data[traffic_data['År'] == 2017]
year_2018 = traffic_data[traffic_data['År'] == 2018]
year_2019 = traffic_data[traffic_data['År'] == 2019]

print("Length of data from 2015:", len(year_2015))
print("Length of data from 2016:", len(year_2016))
print("Length of data from 2017:", len(year_2017))
print("Length of data from 2018:", len(year_2018))
print("Length of data from 2019:", len(year_2019))

In [ ]:
#We can see that the data size is not identical for each year, ie we don't have a value for the traffic at every hour for every year.
#However 2016-2019 seem to have about the same amount of data
def plot_traffic_data_for_year(year_string, year_data):
    x = list(range(0, len(year_data)))
    y = year_data['Volum totalt']
    plt.plot(x,y)
    plt.title(year_string)
    plt.show()
    
plot_traffic_data_for_year("2015", year_2015)
plot_traffic_data_for_year("2016", year_2016)
plot_traffic_data_for_year("2017", year_2017)
plot_traffic_data_for_year("2018", year_2018)
plot_traffic_data_for_year("2019", year_2019)

In [ ]:
#Since it is a bit difficult to tell the average traffic per day with the data represented like this, we can try to get total traffic per day instead of per hour

def get_total_traffic_per_day(traffic_data):
    """
    Gernerates a list of total traffic per day, where the index corresponds to the day number +1.
    
    :param traffic_data: dataframe of traffic data, or a subset of the dataframe (fex data for a year)
    """
    totat_traffic_per_day = np.ones(367)
    for i in range(len(traffic_data)):
        hour_datapoint = traffic_data.iloc[i]
        month_number = hour_datapoint[1]
        day_number = hour_datapoint[2]
        day_of_the_year = datetime.datetime(2016, month_number, day_number).timetuple().tm_yday #retrieve day number
        totat_traffic_per_day[day_of_the_year] += hour_datapoint[6] #add traffic that hour to the total traffic per day
    return totat_traffic_per_day

#generate lists of total traffic per day for the years
total_traffic_per_day_2016 = get_total_traffic_per_day(year_2016)
total_traffic_per_day_2017 = get_total_traffic_per_day(year_2017)
total_traffic_per_day_2018 = get_total_traffic_per_day(year_2018)
total_traffic_per_day_2019 = get_total_traffic_per_day(year_2019)

x_days_total = list(range(0, len(total_traffic_per_day_2016)))


#plotting the traffic pr day
fig = plt.figure(figsize=(20,10))
plt.plot(x_days_total,total_traffic_per_day_2016, 'r-', alpha = 0.7, label = "2016")
plt.plot(x_days_total,total_traffic_per_day_2017, 'b-', alpha = 0.7, label = "2017")
plt.plot(x_days_total,total_traffic_per_day_2018, 'g-', alpha = 0.7, label = "2018")
plt.plot(x_days_total,total_traffic_per_day_2019, 'y-', alpha = 0.7, label = "2019")
plt.title("Total traffic per day")
plt.xticks(np.arange(min(x_days_total), max(x_days_total)+1, 25))
plt.xlabel("Day of the year")
plt.ylabel("Total traffic per day")
plt.legend()
plt.show()

While we can se a consistent trend, we would like to extract a bit more information from the data, such as the day of the year and what weekday each datapoint has. Therefore we construct a list of the traffic data represented in datetime format.

In [ ]:
datetime_representation_of_today = datetime.datetime.today()
print("Datetime representation of today:", datetime_representation_of_today)

def generate_datetime_array_from_traffic_data(traffic_data_input):
    #initialize np array to store datetime of each datapoint in traffic_data
    traffic_data_datetime = np.full(len(traffic_data_input), datetime_representation_of_today)
    for i in range(len(traffic_data_input)):
        data_point = traffic_data_input.iloc[i]
        datetime_representation_of_datapoint = datetime.datetime(data_point[0], data_point[1], data_point[2],data_point[3])
        traffic_data_datetime[i] = datetime_representation_of_datapoint
    return traffic_data_datetime

traffic_data_datetime = generate_datetime_array_from_traffic_data(traffic_data)
print("Datetime represenation of traffic data:")
print(traffic_data_datetime)

In [ ]:
#It is still a bit distracting to see the fluctuation between weekends and weekdays, so we'll take total traffic per week

def get_total_traffic_per_week(traffic_data):
    """
    Gernerates a list of total traffic per day, where the index corresponds to the day number +1.
    
    :param traffic_data: dataframe of traffic data, or a subset of the dataframe (fex data for a year)
    """
    total_traffic_per_week = np.ones(54)
    for i in range(len(traffic_data)):
        hour_datapoint = traffic_data.iloc[i]
        year_number = hour_datapoint[0]
        month_number = hour_datapoint[1]
        day_number = hour_datapoint[2]
        week_number = datetime.datetime(year_number, month_number, day_number).isocalendar()[1]
        total_traffic_per_week[week_number] += hour_datapoint[6] #add traffic that hour to the total traffic per day
    return total_traffic_per_week

#generate lists of total traffic per day for the years
total_traffic_per_week_2016 = get_total_traffic_per_week(year_2016)
total_traffic_per_week_2017 = get_total_traffic_per_week(year_2017)
total_traffic_per_week_2018 = get_total_traffic_per_week(year_2018)
total_traffic_per_week_2019 = get_total_traffic_per_week(year_2019)


x_value_weeks_total = list(range(0, len(total_traffic_per_week_2016)))

#plotting the traffic pr day
fig = plt.figure(figsize=(20,10))
plt.plot(x_value_weeks_total,total_traffic_per_week_2016, 'r-', alpha = 0.7, label = "2016")
plt.plot(x_value_weeks_total,total_traffic_per_week_2017, 'b-', alpha = 0.7, label = "2017")
plt.plot(x_value_weeks_total,total_traffic_per_week_2018, 'g-', alpha = 0.7, label = "2018")
plt.plot(x_value_weeks_total,total_traffic_per_week_2019, 'y-', alpha = 0.7, label = "2019")

# Marking seasonal holidays
plt.axvline(x=9, ymin=0, ymax=60000, color = 'k',alpha = 0.6)
plt.text(9.1,0,'Winter holiday')
plt.axvline(x=41, ymin=0, ymax=60000, color = 'k', alpha = 0.6)
plt.text(41.1,0,'Autumn holiday')
plt.axvline(x=52, ymin=0, ymax=60000, color = 'k', alpha = 0.6)
plt.text(52.1,0,'Christmas holiday')
plt.axvline(x=29, ymin=0, ymax=60000, color = 'k', alpha = 0.6)
plt.text(29.1,0,'Summer holiday')

# Marking easter holidays
plt.axvline(x=12, ymin=0, ymax=60000, color = 'r', alpha = 0.4)
plt.axvline(x=13, ymin=0, ymax=60000, color = 'g', alpha = 0.4)
plt.axvline(x=15, ymin=0, ymax=60000, color = 'b', alpha = 0.4)
plt.axvline(x=16, ymin=0, ymax=60000, color = 'y', alpha = 0.4)
plt.text(16.1,0,'Easter holidays')

# Marking outlier
plt.axvline(x=38, ymin=0, ymax=60000, color = 'c', alpha = 0.4)
plt.text(38.1, 0, "2017 UCI Road World Championships", rotation = "90")
# Marking outlier
plt.axvline(x=25, ymin=0, ymax=60000, color = 'c', alpha = 0.4)
plt.text(25.1, 0, "Missing data", rotation = "90")


# Adding labels
plt.xticks(np.arange(min(x_value_weeks_total), max(x_value_weeks_total)+1, 1))
plt.xlabel("Week of the year")
plt.ylabel("Total traffic per day")
plt.title("Total traffic per week")
plt.legend()
plt.show()

The black vertical lines represent condistent holidays that appear to fall on the same time every year. Unfortunately easter falls on different weeks every year, but we will mark then as "holiday" too. There is some data missing around week 25, but this has no effect even though it looks like it on the graph. Had the graph represented "average traffic per week" there would be no dip in traffic. 

In week 38 in 2017, however, there was a big sports event in Bergen, so this is a true outlier which will have an effect on the model.

Overall we can see that the week number has a lot to say for the fluctuation of data, so we could add it to the list of features.

In [ ]:
weekdays_to_SNTR = [0]*7
weekdays_to_DNP = [0]*7
weekdays_total = [0]*7
is_weekend = []

for i in range(len(traffic_data)):
    data_point = traffic_data.iloc[i]
    weekday = traffic_data_datetime[i].weekday()
    is_weekend.append(int(weekday > 4))  # Adding to one-hot "isWeekend"
    weekdays_to_SNTR[weekday] += data_point[4]
    weekdays_to_DNP[weekday] += data_point[5]
    weekdays_total[weekday] += data_point[6]

print(weekdays_to_SNTR)
print(weekdays_to_DNP)
print(weekdays_total)

day_names = list(calendar.day_name)
fig = plt.figure(figsize=(15,5))
plt.plot(day_names, weekdays_to_DNP, c = 'r', label ="From city center")
plt.plot(day_names, weekdays_to_SNTR, c = 'g', label = "To city center")
plt.title("Mean traffic per weekday to and from city center")
plt.legend()
plt.show()

day_names = list(calendar.day_name)
fig = plt.figure(figsize=(15,5))
plt.plot(day_names, weekdays_total, label ="Total traffic")
plt.title("Mean traffic per weekday in total")
plt.legend()
plt.show()
#TODO: implement one-hot encoding for months and days. Those values have very little to do with each other

In [ ]:
#Now let us look at the traffic for each hour of the day, and compare the traffic in and out of town

def get_total_traffic_per_hour_of_day(string_label):
    y_per_hour = np.zeros(24)
    for i in range(0, 23):
        traffic_per_hour_i = traffic_data[traffic_data['Fra_time'] == i]
        mean_pr_hour = np.mean(traffic_per_hour_i[string_label])
        y_per_hour[i] = mean_pr_hour
    return y_per_hour

y_total_per_hour = get_total_traffic_per_hour_of_day('Volum totalt')
y_to_DNP_per_hour = get_total_traffic_per_hour_of_day('Volum til DNP')
y_to_SNTR_per_hour = get_total_traffic_per_hour_of_day('Volum til SNTR')

x_hour_number = list(range(0, 24))
fig = plt.figure(figsize=(15,5))
plt.plot(x_hour_number,y_total_per_hour, label ="Total traffic")
plt.plot(x_hour_number,y_to_DNP_per_hour, c = 'r', label ="From city center")
plt.plot(x_hour_number,y_to_SNTR_per_hour, c = 'g', label = "To city center")
plt.title("Mean traffic per hour of the day")
plt.xticks(np.arange(min(x_hour_number), max(x_hour_number)+1, 1.0))
plt.legend()
plt.show()


## Feature engineering

### Holiday

In [ ]:
##### Adding holiday feature to data #####

#First we want to make a list of days that we define as "holiday days". We will base this on the holidays identified in the graph above.
holiday_dates = []

#Some helper functions
def getDateRangeFromWeek(p_year,p_week): #sauce: http://mvsourcecode.com/python-how-to-get-date-range-from-week-number-mvsourcecode/
    firstdayofweek = datetime.datetime.strptime(f'{p_year}-W{int(p_week )- 1}-1', "%Y-W%W-%w").date()
    lastdayofweek = firstdayofweek + datetime.timedelta(days=6.9)
    return firstdayofweek, lastdayofweek

def extract_holiday_dates(holdiday_year, holiday_week): #sauce: https://stackoverflow.com/questions/7274267/print-all-day-dates-between-two-dates
    holiday_dates = np.full(7, datetime_representation_of_today) #initialize list to store holiday dated
    firstdate, lastdate =  getDateRangeFromWeek(holdiday_year,holiday_week)
    delta =  lastdate - firstdate       # as timedelta
    for i in range(delta.days + 1):
        holiday_day = firstdate + timedelta(days=i)
        holiday_dates[i] = holiday_day
    return holiday_dates

# The week number of holidays is based on information from Bergen Kommune, timeanddate.no and the graph above
winter_holiday_week_number = 9
summer_holiday_week_number_range = list(range(26, 33))
autumn_holiday_week_number = 41
christmas_holiday_week_number_range = list(range(52, 54))

# Add the holiday dates for years 2015-2019
for year in range(2015, 2020):
    winter_holiday_dates = extract_holiday_dates(year, winter_holiday_week_number)
    autumn_holiday_dates = extract_holiday_dates(year, autumn_holiday_week_number)

    summer_holiday_dates = []
    for week in summer_holiday_week_number_range:
        new_summer_holiday_dates = extract_holiday_dates(year, week)
        summer_holiday_dates = np.concatenate([summer_holiday_dates, new_summer_holiday_dates])

    christmas_holiday_dates = []
    for week in christmas_holiday_week_number_range:
        new_christmas_holiday_dates = extract_holiday_dates(year, week)
        christmas_holiday_dates = np.concatenate([christmas_holiday_dates, new_christmas_holiday_dates])

    holiday_dates = np.concatenate([holiday_dates, winter_holiday_dates, summer_holiday_dates, autumn_holiday_dates, christmas_holiday_dates]) # add holiday dates for the year

holiday_dates = np.concatenate([holiday_dates,
[datetime.date(2015, 12, 28), datetime.date(2015, 12, 29), datetime.date(2015, 12, 30),
 datetime.date(2015, 12, 31), datetime.date(2016, 1, 1), datetime.date(2016, 12, 26),
 datetime.date(2016, 12, 27), datetime.date(2016, 12, 28), datetime.date(2016, 12, 29),
 datetime.date(2016, 12, 30), datetime.date(2016, 12, 31), datetime.date(2017, 1, 1),
 datetime.date(2018, 1, 1), datetime.date(2018, 12, 31), datetime.date(2019, 1, 1),
 datetime.date(2019, 12, 30), datetime.date(2019, 12, 31)]])

# Add the easter weeks separately, as the week of the holiday varies from year to year
holiday_dates = np.concatenate([holiday_dates, 
                                extract_holiday_dates(2016, 13), # dates for easter 2016
                                extract_holiday_dates(2017, 16), # dates for easter 2017
                                extract_holiday_dates(2018, 14), # dates for easter 2018
                                extract_holiday_dates(2019, 16)]) # dates for easter 2019


#Now we have a list of holiday dates, and can generate a hot-one encoded feature array for the dataset.

is_holiday = np.zeros(len(traffic_data)) #Feature array for holidays. 1 if the datapoint is in a holiday, 0 otherwise

#This is a brute force method, which definitely could be improved upon
for i in range(len(traffic_data_datetime)):
    for holiday_date in holiday_dates:
        if traffic_data_datetime[i].date() == holiday_date:
            is_holiday[i] = 1
            continue

traffic_data['is_holiday'] = is_holiday #add "is_holiday" column to traffic_data dataframe

We want to convey the cyclic nature of the hours of the day, and unfortunately the way the hours are represented now gives no indication that the 23rd hour is close to the 0th hour. To solve this we will use two triginometric functions sin((2t*pi)/24) and sin((2t*pi)/24), where t is the hour of the day. We add two features one with sin and the other with cos, because if we only used one of the two, then two hours in the day would have the exact same value, indicating a relationship where there is none.

We do the same with days of the year.

### Cyclical nature of hours and days

In [ ]:
def sin_cos_for_cycical_values(traffic_data_datetime_input, cycle_size, hour_cycle):
    #first we generate a list values for the two functions, so we can just retrieve the values instead of calculate them each time.
    sin_values = np.ones(cycle_size)
    cos_values = np.ones(cycle_size)
    # fill in lists of sin and cos values
    for t in range(cycle_size):
        sin_values[t] = np.sin((2*t*np.pi) / cycle_size)
        cos_values[t] = np.cos((2*t*np.pi) / cycle_size)
    
    #initialize new feature columns
    sin_feature = np.zeros(len(traffic_data_datetime_input)) 
    cos_feature = np.zeros(len(traffic_data_datetime_input))
    #Iterate through all the data, and add the correspongind sin and cos values to the feature columns
    for i in range(len(traffic_data_datetime_input)):
        # retrieve either hour or day number
        if (hour_cycle and cycle_size == 24):
            cyclical_number_of_datapoint = traffic_data_datetime_input[i].hour
        elif(not(hour_cycle) and cycle_size == 367):
            cyclical_number_of_datapoint = traffic_data_datetime_input[i].timetuple().tm_yday
        else:
            print("Invalid cycle size:", cycle_size)
            print("TOOOOO")
            return
        sin_feature[i] = sin_values[cyclical_number_of_datapoint]
        cos_feature[i] = cos_values[cyclical_number_of_datapoint]
    return sin_feature, cos_feature
        
traffic_data['sin_hour'], traffic_data['cos_hour'] = sin_cos_for_cycical_values(traffic_data_datetime, 24, True)
traffic_data['sin_day'], traffic_data['cos_day'] = sin_cos_for_cycical_values(traffic_data_datetime, 367, False)

### Daylight saving time

In [ ]:
traffic_data['is_weekend'] = is_weekend #add "is_weekend" column to traffic_data dataframe
#DST = daylight saving time
#Generate a column for the DST feature, with 1 indicating the datapoint is in DST, and 0 if it is not
DST = []
DST_switch = False
for i in range(len(traffic_data)):
    data_point = traffic_data.iloc[i]
    # TODO: Abstract/refactor the line under
    if data_point[0] == 2016 and data_point[1] == 3 and data_point[2] == 27 and data_point[3] == 2:
        DST_switch = True
    elif data_point[0] == 2016 and data_point[1] == 10 and data_point[2] == 30 and data_point[3] == 3:
        DST_switch = False
    elif data_point[0] == 2017 and data_point[1] == 3 and data_point[2] == 26 and data_point[3] == 2:
        DST_switch = True
    elif data_point[0] == 2017 and data_point[1] == 10 and data_point[2] == 29 and data_point[3] == 3:
        DST_switch = False
    elif data_point[0] == 2018 and data_point[1] == 3 and data_point[2] == 25 and data_point[3] == 2:
        DST_switch = True
    elif data_point[0] == 2018 and data_point[1] == 10 and data_point[2] == 28 and data_point[3] == 3:
        DST_switch = False
    elif data_point[0] == 2019 and data_point[1] == 3 and data_point[2] == 31 and data_point[3] == 2:
        DST_switch = True
    elif data_point[0] == 2019 and data_point[1] == 10 and data_point[2] == 27 and data_point[3] == 3:
        DST_switch = False
    DST.append(int(DST_switch))

traffic_data['is_DST'] = DST #add DST column to traffic_data dataframe

# Model selection

We will be evaluating some models and selecting the most appropriate for:
- Predicting TOTAL traffic volume
- Predicting traffic TO city center
- Predicting traffic FROM city center

These models will be named TOTAL, TO and FROM respectively

## Model selection for predicting TOTAL traffic volume

### Baseline using linear regression -TOTAL

In [ ]:
#TODO

### Random forests -TOTAL

In [ ]:
#Use TimeSeriesSplit to split the data into train and test to get accuracies
X = traffic_data[["År", "Måned", "Dag", "Fra_time", "sin_hour", "cos_hour", "sin_day", "cos_day", "is_holiday", "is_weekend", "is_DST"]].values
y = traffic_data["Volum totalt"].values
#Split data into train_val and test data. Do not shuffle the data as it is timeseries data
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

number_of_splits = 5
tscv = TimeSeriesSplit(number_of_splits)


print("###\tRandom Forest\t###\nTotal number of splits:", number_of_splits)
split_number = 1

list_of_coefficients_of_determination = [] #list to keep track of coefficients of determination
for train_index, val_index in tscv.split(X_train_val):
    print("Split number:", split_number)
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]

    forest = ens.RandomForestRegressor()
    forest.fit(X_train,  y_train)
    forest_predict = forest.predict(X_val) 
    #forest_MSE = mean_squared_error(y_test, forest_predict)
    print("Mean Absolute Error: %.2f" %mean_absolute_error(y_val, forest_predict))
    coefficient_of_determination = r2_score(y_val, forest_predict)
    list_of_coefficients_of_determination.append(coefficient_of_determination)
    print('Coefficient of determination: %.2f' %coefficient_of_determination)
    split_number += 1    
    print("")
print('Mean coefficient of determination: %.2f' %np.mean(list_of_coefficients_of_determination)) #might  be some sklearn function that does this for us

### Regression decision tree - TOTAL

In [ ]:

print("\n\n###\tRegression Decision Tree\t###\nTotal number of splits:", number_of_splits)
best_mean_coeff_det = -1
best_mean_coeff_det_n = 0
for i in range(1, 30):
    split_number = 1
    list_of_coefficients_of_determination = [] #list to keep track of coefficients of determination
    for train_index, val_index in tscv.split(X_train_val):
        #print("Split number:", split_number)
        X_train, X_val = X_train_val[train_index], X_train_val[val_index]
        y_train, y_val = y_train_val[train_index], y_train_val[val_index]

        tree = DecisionTreeRegressor(max_depth=i)
        tree.fit(X_train,  y_train)
        tree_predict = tree.predict(X_val)
        #tree_MSE = mean_squared_error(y_test, tree_predict)
        #print("Mean Absolute Error: %.2f" %mean_absolute_error(y_val, tree_predict))
        coefficient_of_determination = r2_score(y_val, tree_predict)
        list_of_coefficients_of_determination.append(coefficient_of_determination)
        print('Coefficient of determination: %.2f' %coefficient_of_determination)
        split_number += 1
        print("")
    print("Max tree depth", i)
    mean_coeff_det = np.mean(list_of_coefficients_of_determination)
    if mean_coeff_det > best_mean_coeff_det:
        best_mean_coeff_det = mean_coeff_det
        best_mean_coeff_det_n = i
    print('Mean coefficient of determination: %.2f\n\n' %mean_coeff_det) #might  be some sklearn function that does this for us
print("Best max depth is", best_mean_coeff_det_n, 'with Mean coefficient of determination: %.2f' %best_mean_coeff_det)

### Neural network - TOTAL

In [ ]:
print("\n\n###\tNeural Network\t###\nTotal number of splits:", number_of_splits)

norm_td=(traffic_data-traffic_data.min())/(traffic_data.max()-traffic_data.min()) #TODO: experimental
X = norm_td[["År", "Måned", "Dag", "Fra_time", "sin_hour", "cos_hour", "sin_day", "cos_day", "is_holiday", "is_weekend", "is_DST"]].values
y = norm_td["Volum totalt"].values
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

split_number = 1

list_of_coefficients_of_determination = [] #list to keep track of coefficients of determination
for train_index, val_index in tscv.split(X_train_val):
    print("Split number:", split_number)
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    y_train, y_val = y_train_val[train_index], y_train_val[val_index]

    # Optimal number of neurons should be between 300 and 1500 (one layer is enough, apparently)
    nn = MLPRegressor(hidden_layer_sizes=(90, 90, 90, 90, 90, 90, 90, 90), random_state=1, max_iter=400)  # Default Relu with Adam cost optimizer
    nn.fit(X_train,  y_train)
    nn_predict = nn.predict(X_val)
    #nn_MSE = mean_squared_error(y_test, nn_predict)
    print("Mean Absolute Error: %.4f" %mean_absolute_error(y_val, nn_predict))
    coefficient_of_determination = r2_score(y_val, nn_predict)
    list_of_coefficients_of_determination.append(coefficient_of_determination)
    print('Coefficient of determination: %.4f' %coefficient_of_determination)
    split_number += 1
    print("")
print('Mean coefficient of determination: %.4f' %np.mean(list_of_coefficients_of_determination)) #might  be some sklearn function that does this for us

## Testing of selected model for predicting TOTAL traffic volume

In [ ]:
# TODO: test selected model w testdata

## Model selection for predicing traffic TO city center

In [ ]:
#TODO: same as with total volume, just with other traffic

### Baseline with linear regression - TO

In [ ]:
# TODO

### Random forests - TO

In [ ]:
# TODO

### Regression trees - TO

In [ ]:
# TODO

### Neural networks - TO

In [ ]:
# TODO

### Testing of selected model for predicting traffic TO city center

In [ ]:
# TODO

### Predict traffic TO city center

In [ ]:
# TODO

### Random forests - FROM

In [ ]:
# TODO

### Regression trees - FROM

In [ ]:
# TODO

### Neural networks - FROM

In [ ]:
# TODO

### Testing of selected model for predicting traffic FROM city center

In [ ]:
# TODO

### Predict traffic FROM city center

In [ ]:
# TODO




## Differneces in predicting traffic TO and FROM city center
TODO: write

## Performance of models on 2020 data

To test the perfomance of our trained models on 2020 data, we will first feature engineer our 2020 data in the same we as we did previously, and thereafter use it as test data to evaluate our TOTAL, TO and FROM models.

In [ ]:
traffic_data_2020 = pd.read_csv("data_2020.csv")

#Generate datetime array for 2020 traffic data
traffic_data_2020_datetime = generate_datetime_array_from_traffic_data(traffic_data_2020)

traffic_data_2020['sin_hour'], traffic_data_2020['cos_hour'] = sin_cos_for_cycical_values(traffic_data_2020_datetime, 24, True)
traffic_data_2020['sin_day'], traffic_data_2020['cos_day'] = sin_cos_for_cycical_values(traffic_data_2020_datetime, 367, False)

print(traffic_data_2020)


In [ ]:
# Regression Decision Tree
days = 60
hours = days * 24
x = traffic_data[["År", "Måned", "Dag", "Fra_time", "is_weekend", "is_DST"]].values
y = traffic_data["Volum totalt"].values

tree = DecisionTreeRegressor(max_depth=10)
tree.fit(x, y)

test_1 = traffic_data[["År", "Måned", "Dag", "Fra_time", "is_weekend", "is_DST"]].values[hours:2*hours]
result_1 = tree.predict(test_1)

fig = plt.figure(figsize=(15,5))
x_hour = list(range(hours))
plt.plot(x_hour, result_1)
plt.scatter(x_hour, traffic_data["Volum totalt"].values[hours:2*hours])
plt.show()